In [1]:
import pandas as pd
import camelot
import re
import numpy as np

In [153]:
# reg1: sträng börjar med en siffra eller "<" eller ">".
reg1 = re.compile(r'^\d|<|>')

reg11 = re.compile(r'\d.$')

# reg2: sträng börjar med någonting annat än siffra.
reg2 = re.compile(r'^\D')

# reg3: variabelnamn.
reg3 = re.compile(r'^Z\d|^PZ\d')

# reg4: sträng är en siffra med en minst en decimal.
reg4 = re.compile(r'\d,|\d\.')

# reg5: sträng innehåller bokstäver.
reg5 = re.compile(r'[a-zA-Z]')

delar = []

for i in range(50, 51):
    # Läs in sidan i från LNU:s kodbok
    tables = camelot.read_pdf('LNU{}.pdf'.format(i))
    
    print(i)
    # Gör om "tabellen" på sidan till en DataFrame. Vissa sidor, t.ex. 36, har ingen tabell alls. 
    # Kolla därför att tabell finns.
    if tables.n == 0:
        # Om tabell "inte finns" skapas en df med en rad som säger sidbrytning i alla kolumner.
        A = pd.DataFrame(data = [['sidbrytning', 'sidbrytning', 'sidbrytning', 'sidbrytning']],
                 columns = ['Kod', 'Svarstext', 'Frekvens', 'Variabel'])
        # Denna df läggs sedan till i listan delar.
        delar.append(A)
    else:
        df = tables[0].df
        rutor = [df.loc[i][0].split('\n') for i in df.index]
        

        # j är en ruta i kodboken.
        for j in range(len(rutor)):
            # print('j:' + str(j))

            namn = []
            koder = []
            texter = []
            frekvenser = []

            # k är en sträng i en ruta.
            for k in range(len(rutor[j])):
                # print('k:' + str(k))

                kod = rutor[j][k]

                # Ifall variabeln saknar frekvenstabell eller hamnar i sidbrytning (Z112) är antalet k väldigt litet
                # och k+1 och k+2 riskerar att hamna utanföre "index range". Andra variabler (Z14) har flera extra blanka
                # k i slutet pga en förklarande text som kommer längst ner i rutan. I sådana fall blir siffror som egentligen
                # inte är koder till koder Breaka därför närhelst k är tre steg från slutet. 
                if (k > len(rutor[j]) - 3):
                    break

                text = rutor[j][k+1]
                frekvens = rutor[j][k+2]

                # Dessa villkor blir så långa. Det blir lättare att läsa förkortningar.
                # t1: sträng börjar med siffra eller "<"
                t1 = bool(reg1.findall(kod))
                
                # t11: sträng slutar med en siffra (typ... se reg11 för detaljer)
                t11 = bool(reg11.findall(kod))

                # t2: Strängen innehåller en siffra som följs av en punkt.
                t2 = bool(reg4.findall(rutor[j][k]))

                # t2: Nästa sträng innehåller en siffra som följs av en punkt.
                t3 = bool(reg4.findall(rutor[j][k+1]))
                
                # t4: k-1 innehåller bokstäver eller k-2 innehåller bokstäver
                t4 = bool(reg5.findall(rutor[j][k-1])) or bool(reg5.findall(rutor[j][k-2]))
                                
                # t5: text (k+1) innehåller bokstäver eller text börjar med någonting annat än en siffra
                t5 = (bool(reg5.findall(text)) == True) or (bool(reg2.findall(text)) == True)
                
                # Den första sträng som fångas av reg3 måste vara variabelnamnet.
                if (bool(reg3.findall(kod)) == True) & (len(namn) == 0):
                    namn.append(kod)

                # En kod måste inledas med en siffra (eller "<" eller ">") (t1) samt samt sluta med en siffra (t11).
                if (t1 == True) & (t11 == True):
                    # Ifall detta är den första koden
                    if (len(koder) == 0):
                        #print('första koden')

                        # t5: Ifall text (k+1) innehåller bokstav eller text börjar med någonting annat än en siffra
                        if t5 == True:
                            koder.append(kod)
                            texter.append(text)
                            frekvenser.append(frekvens)
                        # Ifall text börjar med siffra är det snarare en frekvens.
                        else:
                            koder.append(kod)
                            texter.append(' ')
                            frekvenser.append(text)

                    # Ifall listan med koder inte är tom
                    # KOLLA DOK HÄR

                    elif (t4 == False) & (t2 == False) & (t3 == False):
                        #print('inte första koden')
                        if bool(reg2.findall(text)) == True:
                            koder.append(kod)
                            texter.append(text)
                            frekvenser.append(frekvens)
                        else:
                            koder.append(kod)
                            texter.append(' ')
                            frekvenser.append(text)
                            
            # Variabler som Z18 har ingen frekvenstabell. Lägg då in 'saknas' i dataframen
            if len(koder) == 0:
                koder.append('saknas')
                texter.append('saknas')
                frekvenser.append('saknas')    
                    
                #print('koder: ' + str(koder), 'texter: ' + str(texter), 'frekvenser: ' + str(frekvenser), sep = '\n')
            if len(namn) == 0:
                namn.append('sidbrytning')
            #print('namn: ' + namn[0])

            # Innan nästa ruta i kodboken (j) så skapas en df med koder, texter och frekvenser som kolumner.
            A = pd.DataFrame.from_dict({'Kod': koder, 'Svarstext': texter, 'Frekvens': frekvenser})

            # Lägg till en kolumn för variabelnamn.
            A['Variabel'] = namn[0]

            #Lägg till den färdiga frekvenstabellen i listan "delar".
            delar.append(A)
    

50


Här är en dataframe när loopen *inte* breakat vid total. Istället har en stannat när strängarna i rutan tar slut. Här tas alltså det mesta med.

In [154]:
LNU2 = pd.concat(delar)

In [155]:
LNU2.head(50)

,Kod,Svarstext,Frekvens,Variabel
0,1,Ja,1150,Z48
1,2,Nej,621,Z48
2,9,Svar saknas,2,Z48
0,1,Ja,2439,Z49
1,2,Nej,99,Z49
2,3,Båda utländska,718,Z49
3,9,Svar saknas,9,Z49
0,1,Svensk,3566,Z50
1,2,"Dansk, norsk",73,Z50
2,3,Finsk,135,Z50


**TEST**

In [127]:
tables = camelot.read_pdf('LNU44.pdf')

if tables.n == 0:
    print('INGEN tabell hehe')
else:
    df = tables[0].df
    
rutor = [df.loc[i][0].split('\n') for i in df.index]

In [128]:
rutor[1]

['Z38 ',
 'BOKOSTNAD – EJ FÖRESTÅNDARE ',
 'X46, Y122, ',
 'U72, V61 ',
 'Betalar du själv något för bostaden? Hur mycket per månad (kr)? Variabeln är här kategoriindelad. ',
 'Tabellen gäller för Z37 NE 1.  ',
 '0 ',
 ' ',
 '182 ',
 '50,0 ',
 '500-999 ',
 ' ',
 '6 ',
 '1,6 ',
 '1000-1999 ',
 ' ',
 '20 ',
 '5,5 ',
 '1500-1999 ',
 ' ',
 '23 ',
 '6,3 ',
 '2000-2499 ',
 ' ',
 '31 ',
 '8,5 ',
 '2500-2999 ',
 ' ',
 '24 ',
 '6,6 ',
 '3000-3499 ',
 ' ',
 '22 ',
 '6,0 ',
 '3500-3999 ',
 ' ',
 '18 ',
 '4,9 ',
 '4000-4499 ',
 ' ',
 '13 ',
 '3,6 ',
 '4500-4599 ',
 ' ',
 '6 ',
 '1,6 ',
 '>5000 ',
 ' ',
 '19 ',
 '5,2 ',
 ' ',
 'Totalt ',
 '364 ',
 '100 ',
 '888888 ',
 'Vet ej ',
 '3 ',
 ' ',
 '999999 ',
 'Svar saknas ',
 '5 ',
 ' ',
 ' ',
 'Ej aktuellt ',
 '4043 ',
 ' ',
 ' ',
 ' ',
 '4415']

In [134]:
len(rutor[1])

69

In [137]:
j = 1
k = 46

In [138]:
rutor[j][k]

'>5000 '

In [148]:
reg1 = re.compile(r'^\d|>')

In [143]:
# Dessa villkor blir så långa. Det blir lättare att läsa förkortningar.
# t1: sträng börjar med siffra eller "<"
t1 = bool(reg1.findall(kod))
                
# t11: sträng slutar med en siffra (typ... se reg11 för detaljer)
t11 = bool(reg11.findall(kod))

# t2: Strängen innehåller en siffra som följs av en punkt.
t2 = bool(reg4.findall(rutor[j][k]))

# t2: Nästa sträng innehåller en siffra som följs av en punkt.
t3 = bool(reg4.findall(rutor[j][k+1]))
                
# t4: k-1 innehåller bokstäver eller k-2 innehåller bokstäver
t4 = bool(reg5.findall(rutor[j][k-1])) or bool(reg5.findall(rutor[j][k-2]))
                                
# t5: både k+1 och k+2 är whitespace
t5 = (bool(reg5.findall(text)) == True) or (bool(reg2.findall(text)) == True)

In [144]:
(t4 == False) & (t2 == False) & (t3 == False)

True

In [149]:
bool(reg1.findall(rutor[1][46]))

True

In [86]:
k == len(rutor[0])-2

True

In [78]:
rutor[0][60]

IndexError: list index out of range

In [28]:
j = 1
k = 10

In [29]:
rutor[j][k]

'2 '

In [30]:
t4 = bool(reg5.findall(rutor[j][k-1])) or bool(reg5.findall(rutor[j][k-2]))
                
t5 = (rutor[j][k+1] == ' ') & (rutor[j][k+2] == ' ')

In [31]:
t4

False

In [20]:
bool(reg5.findall(rutor[j][k-1])) or bool(reg5.findall(rutor[j][k-2]))

True

In [18]:
bool(reg5.findall(rutor[j][k-2]))

False

In [19]:
True or False

True

 
 
* Z50 har "okänd" efter "total".  
***  
Problemet verkar vara att villkoren t2 och t3 är för svaga. De släpper fram för mycket.  
1. För att undanta de två siffror följer efter "total" borde ett villkor som säger: ifall k-1 elelr k-2 är lika med total --> undanta  
2. För att undanta resterande oönskade siffror efter total borde ett villkor som säger: ifall k+1 och k+2 är whitespace --> undanta  
***  


Ibland avslutas en ruta med en sträng som är en fotnot. I dessa falla har jag inte lyckas undvika att den sista siffran läggs till som kod. Lyckligtvis är denna siffra alltid 4415 (antalet i kohorten?) och därför är det enkelt att i efterhand välja ut alla rader vars kod är just 4415 och ta bort dem.

Z24 läses in fel av camelot. Det blir en förskjutning av koderna. Alternativ 1 har blivit alternativ 2 osv. Korrigera detta i efterhand genom att se vilka koder som är orimligt stora i kod-kolumnen.